# Chest X-ray Image Report Generation (CXIRG)

## Install Required Modules

In [35]:
!pip install openpyxl
!pip install pandas
!pip install pillow
!pip install pytorch-ignite
!pip install scikit-learn
!pip install torch
!pip install transformers

## Import Required Modules

In [36]:
import os
import random
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import nltk

from ignite.metrics import Rouge
from pandas.core.common import random_state
from PIL import Image
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, VisionEncoderDecoderModel, ViTImageProcessor
from typing import Any, Dict, List
from torch.utils.data import DataLoader
from datasets import load_metric

## Set The Random Seed

In [37]:
seed = 48763

np.random.seed(seed=seed, )

random_state(state=seed, )

random.seed(a=seed, )

torch.manual_seed(seed=seed, )
torch.cuda.manual_seed(seed=seed, )
torch.cuda.manual_seed_all(seed=seed, )
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Set The Device & Initialize Models

In [38]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia", "emilyalsentzer/Bio_ClinicalBERT").to(device)

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

print(tokenizer.all_special_tokens)

tokenizer.add_special_tokens({
    "bos_token": tokenizer.cls_token,
    "eos_token": tokenizer.sep_token,
    "pad_token": tokenizer.pad_token,
    #"bos_token": "<|beginoftext|>",
    #"pad_token": "<|pad|>",
})

model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
#model.generation_config.decoder_start_token_id = tokenizer.cls_token_id

model.decoder.resize_token_embeddings(len(tokenizer))

processor = ViTImageProcessor.from_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia")

c:\Users\kartg\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at nickmuchi/vit-finetuned-chest-xray-pneumonia and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.enc

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


## The CXIRG Dataset

In [39]:
class CXIRGDataset(Dataset):
    def __init__(self, data: List[Dict[str, Any]]) -> None:
        super(CXIRGDataset, self).__init__()
        self.data = data

    def __getitem__(self, index: int) -> Dict[str, Any]:
        return self.data[index]

    def __len__(self) -> int:
        return len(self.data)

## The Collate Function for The DataLoader

In [40]:
max_length = 256

def train_dl_collate_fn(one_batch_data: List[Dict[str, Any]]) -> Dict[str, Any]:
    names, pixel_values, reports = [], [], []

    for one_data in one_batch_data:
        names.append(one_data["name"])
        pixel_values.append(one_data["image"])
        reports.append(one_data["text"])

    # Process images
    pixel_values = processor(images=pixel_values, return_tensors="pt").pixel_values
    
    # Tokenize reports
    label_ids = tokenizer(
        reports,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).input_ids

    return {
        "names": names,
        "pixel_values": pixel_values.to(device, dtype=torch.float32),
        "label_ids": label_ids.to(device, dtype=torch.int64)
    }

def valid_dl_collate_fn(one_batch_data: List[Dict[str, Any]]) -> Dict[str, Any]:
    names, pixel_values, reports = [], [], []

    for one_data in one_batch_data:
        names.append(one_data["name"])
        pixel_values.append(one_data["image"])
        reports.append(one_data["text"])

    # Process images
    pixel_values = processor(images=pixel_values, return_tensors="pt").pixel_values

    # Tokenize reports
    label_ids = tokenizer(
        reports,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).input_ids

    return {
        "names": names,
        "pixel_values": pixel_values.to(device, dtype=torch.float32),
        "label_ids": label_ids.to(device, dtype=torch.int64)
    }


## Load The Train & Validation Data

In [41]:
report_path = "CXIRG_Data/train_data/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "CXIRG_Data/train_data/images"

train_data = []

for image_name in os.listdir(image_dir_path):
    image_path = os.path.join(image_dir_path, image_name)
    
    with Image.open(image_path) as img:
        if img.mode != "RGB":
            img = img.convert("RGB")
        text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

        train_data.append({
            "name": image_name[:13],
            "image": img,
            "text": text
        })


train_dataset = CXIRGDataset(train_data)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=train_dl_collate_fn)

for batch in train_dataloader:
    print(batch)

{'names': ['NLP_CHEST_001', 'NLP_CHEST_048', 'NLP_CHEST_060', 'NLP_CHEST_094'], 'pixel_values': tensor([[[[-0.1843, -0.4118, -0.4118,  ..., -0.9137, -0.9137, -0.9137],
          [-0.5686, -0.4667, -0.3804,  ..., -0.7098, -0.6235, -0.5765],
          [-0.6549, -0.5216, -0.3490,  ..., -0.4902, -0.5059, -0.4196],
          ...,
          [ 0.2941,  0.3412,  0.3882,  ..., -0.9529, -0.9451, -0.9373],
          [ 0.2706,  0.3020,  0.3804,  ..., -0.9451, -0.9451, -0.9373],
          [ 0.2471,  0.2863,  0.3490,  ..., -0.9451, -0.9373, -0.9294]],

         [[-0.1843, -0.4118, -0.4118,  ..., -0.9137, -0.9137, -0.9137],
          [-0.5686, -0.4667, -0.3804,  ..., -0.7098, -0.6235, -0.5765],
          [-0.6549, -0.5216, -0.3490,  ..., -0.4902, -0.5059, -0.4196],
          ...,
          [ 0.2941,  0.3412,  0.3882,  ..., -0.9529, -0.9451, -0.9373],
          [ 0.2706,  0.3020,  0.3804,  ..., -0.9451, -0.9451, -0.9373],
          [ 0.2471,  0.2863,  0.3490,  ..., -0.9451, -0.9373, -0.9294]],

      

In [42]:
report_path = "CXIRG_Data/valid_data/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "CXIRG_Data/valid_data/images"

valid_data = []


for image_name in os.listdir(image_dir_path):
    image_path = os.path.join(image_dir_path, image_name)
    
    with Image.open(image_path) as img:
        if img.mode != "RGB":
            img = img.convert("RGB")

        text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

        valid_data.append({
            "name": image_name[:13],
            "image": img,
            "text": text
        })

valid_dataset = CXIRGDataset(valid_data)

valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=False, collate_fn=valid_dl_collate_fn)

for batch in valid_dataloader:
    print(batch)

{'names': ['NLP_CHEST_002', 'NLP_CHEST_004', 'NLP_CHEST_011', 'NLP_CHEST_015'], 'pixel_values': tensor([[[[-0.4980, -0.4824, -0.4745,  ..., -0.7255, -0.6549, -0.5059],
          [-0.3961, -0.3961, -0.3882,  ..., -0.7333, -0.7255, -0.6863],
          [-0.3882, -0.3882, -0.3882,  ..., -0.7412, -0.7333, -0.7255],
          ...,
          [-0.6863, -0.6863, -0.7020,  ..., -1.0000, -0.9922, -0.9922],
          [-0.6784, -0.6941, -0.7020,  ..., -1.0000, -0.9922, -0.9922],
          [-0.6706, -0.6784, -0.6863,  ..., -1.0000, -0.9922, -0.9922]],

         [[-0.4980, -0.4824, -0.4745,  ..., -0.7255, -0.6549, -0.5059],
          [-0.3961, -0.3961, -0.3882,  ..., -0.7333, -0.7255, -0.6863],
          [-0.3882, -0.3882, -0.3882,  ..., -0.7412, -0.7333, -0.7255],
          ...,
          [-0.6863, -0.6863, -0.7020,  ..., -1.0000, -0.9922, -0.9922],
          [-0.6784, -0.6941, -0.7020,  ..., -1.0000, -0.9922, -0.9922],
          [-0.6706, -0.6784, -0.6863,  ..., -1.0000, -0.9922, -0.9922]],

      

## Set The Hyperparameters & Initialize The Optimizer, Dataloaders and Evaluation Metric

In [43]:
lr = 1e-5
epochs = 30
optimizer = AdamW(params=model.parameters(), lr=lr)

train_batch_size = 1
valid_batch_size = 1
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=train_dl_collate_fn
)
valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=valid_batch_size,
    shuffle=False,
    collate_fn=valid_dl_collate_fn
)

rouge = Rouge(variants=["L", 2], multiref="best")

## The Evaluation Function

In [44]:
def evaluate(model: VisionEncoderDecoderModel, epoch: int) -> Dict[str, float]:
    model.eval()
    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"Evaluating Epoch: {epoch + 1}")

    #rouge = load_metric("rouge")

    with torch.no_grad():
        for batch in pbar:
            names = batch["names"]
            pixel_values = batch["pixel_values"]
            label_ids = batch["label_ids"]

            predictions = model.generate(
                pixel_values, 
                decoder_start_token_id=model.config.decoder_start_token_id,
                max_length=256,
                num_beams=4,     
                early_stopping=True
            )

            _predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            _labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

            print(f"Names      : {names}")
            print(f"Predictions: {_predictions}")
            print(f"Labels     : {_labels}")
            print()

            for _prediction, _label in zip(_predictions, _labels):
                split_prediction = _prediction.split()
                split_label = _label.split()

                rouge.update(([split_prediction], [[split_label]]))

    return rouge.compute()


In [45]:
epochs = 10
min_loss = float("inf")

# 訓練和評估循環
for epoch in range(epochs):
    model.train()

    pbar = tqdm(train_dataloader)
    pbar.set_description(f"Training Epoch [{epoch + 1} / {epochs}]")

    loss_list = []
    average_loss = 0

    for batch in pbar:
        optimizer.zero_grad()

        pixel_values = batch["pixel_values"]
        label_ids = batch["label_ids"]

        outputs = model(
            pixel_values=pixel_values,
            labels=label_ids
        )
        loss = outputs.loss

        loss_list.append(loss.item())
        average_loss = np.mean(np.array(loss_list))
        loss.backward()
        optimizer.step()

        pbar.set_postfix(loss=average_loss)

    if average_loss < min_loss:
        min_loss = average_loss
        torch.save(model.state_dict(), f"outputs/best_checkpoint.pt")

    print(f"Rouge-2 score on epoch {epoch+1}:", evaluate(model=model, epoch=epoch))

Evaluating Epoch: 1:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\kartg\anaconda3\envs\myenv\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluating Epoch: 1:  10%|█         | 1/10 [00:01<00:14,  1.66s/it]

Names      : ['NLP_CHEST_002']
Predictions: ['...................']
Labels     : ['chest pa view shows : partial atelectasis at left lower lung field. left pleural effusion. borderline heart size. atherosclerotic change of aortic knob. s / p port - a implantation via left subclavian vein. intraperitoneal port catheter. surgical clips at ruq of abdomen.']



Evaluating Epoch: 1:  20%|██        | 2/10 [00:03<00:12,  1.57s/it]

Names      : ['NLP_CHEST_004']
Predictions: ['...................']
Labels     : ['chest pa view show : impression : - s / p rll wedge resection. - bilateral lungs metastasis. - left lower lung subsegmental atelectasis. - increased infiltrations in both lungs. - blunting right cp angle. - tortuous atherosclerotic aorta. - scoliosis, djd and osteoporosis of spine. - compression fracture of l1.']



Evaluating Epoch: 1:  30%|███       | 3/10 [00:04<00:10,  1.55s/it]

...

Names      : ['NLP_CHEST_059']
Predictions: ['']
Labels     : ['supine chest ap view shows : pigtail catheter at right lower chest. decreased amount of right pleural effusion. air in the right lower pleural space. poor lung expansion. mild subcutaneous emphysema at right chest wall. right pleural thickening. ill - defined masses at right perihilar region. normal heart size.']



Evaluating Epoch: 10:  90%|█████████ | 9/10 [00:07<00:00,  1.23it/s]

Names      : ['NLP_CHEST_071']
Predictions: ['chest ap view shows : impression : - increased both lung markings. - the heart size is']
Labels     : ['chest film shows : impression : - bilateral lung infiltrations. - suspect right lower lung patch. blunting right cp angle. - tortuous atherosclerotic aorta. - scoliosis, djd and osteoporosis of spine. compression fracture of t12. old fracture of left ribs. - s / p fixation in l - spine. - s / p tracheostomy and ng tube.']



Evaluating Epoch: 10: 100%|██████████| 10/10 [00:08<00:00,  1.20it/s]

Names      : ['NLP_CHEST_085']
Predictions: ['chest ap view shows : impression : - increased infiltrates in both lungs. -']
Labels     : ['chest x ray : - right pneumothorax with pleural effusion, status post drainge tube placement. - extensive subcutaneous emphysema from neck to right side thoracic cage. - increased right hilar opacity. - atherosclerosis and tortuous aorta. - obscured bilateral costophrenic angles. - spondylosis and mild scoliosis of thoracolumbar spine. - oa of right glenohumeral joint. - generalized osteopenia.']

Rouge-2 score on epoch 9: {'Rouge-L-P': 0.20536853398579408, 'Rouge-L-R': 0.0608688583305246, 'Rouge-L-F': 0.0608688583305246, 'Rouge-2-P': 0.1043890064184182, 'Rouge-2-R': 0.02879294411421789, 'Rouge-2-F': 0.02879294411421789}
